# Lab 5 Threat hunting

## Diego Franco 20240

In [2]:
import pandas as pd
from pandas import json_normalize
import json
import random
from clasificador import *
import csv
import whois

### Leer el json con la data

In [3]:
import json

# Paso 1: Definir la ubicación del archivo
archivo = "Archivos/large_eve.json"

# Paso 2: Cargar el archivo JSON en una lista de diccionarios
with open(archivo, 'r') as f:
    data = [json.loads(line) for line in f]

# Paso 3: Obtener la cantidad total de registros
cantidad_total_registros = len(data)
print("Cantidad total de registros:", cantidad_total_registros)



Cantidad total de registros: 746909


### Obtener unicamente los DNS

In [4]:
# Filtrar la lista de diccionarios para obtener registros DNS
registros_dns = [registro for registro in data if 'event_type' in registro and 'dns' in registro['event_type'].lower()]

# Obtener la cantidad de registros DNS
cantidad_registros_dns = len(registros_dns)
print("Cantidad de registros DNS:", cantidad_registros_dns)



Cantidad de registros DNS: 15749


### Mostrar 2 registros

In [5]:
# Seleccionar dos registros aleatorios de registros_dns
dos_registros_cualesquiera = random.sample(registros_dns, 2)

# Imprimir los dos registros seleccionados
for registro in dos_registros_cualesquiera:
    print(registro)

{'timestamp': '2017-07-22T18:29:17.502533-0500', 'flow_id': 333716841671229, 'pcap_cnt': 504853, 'event_type': 'dns', 'vlan': 150, 'src_ip': '192.168.207.4', 'src_port': 53, 'dest_ip': '192.168.205.188', 'dest_port': 39689, 'proto': 'UDP', 'dns': {'type': 'answer', 'id': 2970, 'rcode': 'NXDOMAIN', 'rrname': '<root>', 'rrtype': 'SOA', 'ttl': 20864}}
{'timestamp': '2017-07-22T18:29:17.763025-0500', 'flow_id': 1916096610149321, 'pcap_cnt': 505384, 'event_type': 'dns', 'vlan': 150, 'src_ip': '192.168.207.4', 'src_port': 53, 'dest_ip': '192.168.205.188', 'dest_port': 54263, 'proto': 'UDP', 'dns': {'type': 'answer', 'id': 41704, 'rcode': 'NXDOMAIN', 'rrname': '<root>', 'rrtype': 'SOA', 'ttl': 20864}}


### Convertir a data frame

In [6]:
# Normalizar la información contenida en los JSON anidados y asignarla a un DataFrame
df_normalized = json_normalize(registros_dns)

# Mostrar la forma (shape) del DataFrame
print("Shape del DataFrame:", df_normalized.shape)

Shape del DataFrame: (15749, 18)


In [7]:
# Filtrar los registros DNS tipo A
registros_dns_tipo_a = df_normalized[df_normalized['dns.rrtype'] == 'A']

print("Shape del DataFrame con registros DNS tipo A:", registros_dns_tipo_a.shape)

Shape del DataFrame con registros DNS tipo A: (2849, 18)


### Filtrar dominios unicos

In [8]:
dominios_unicos = registros_dns_tipo_a['dns.rrname'].unique()
print("Cantidad de dominios únicos:", len(dominios_unicos))

Cantidad de dominios únicos: 177


### TLD

In [9]:
def obtener_tld(dominio):
    """
    Función para obtener el TLD (Dominio de Nivel Superior) de un dominio dado.
    
    Args:
    - dominio (str): El dominio del cual se quiere obtener el TLD.
    
    Returns:
    - str: El TLD del dominio.
    """
    partes = dominio.split('.')
    for i in range(len(partes) - 1, 0, -1):
        # Recorrer las partes del dominio en orden inverso
        tld_potencial = '.'.join(partes[i:])
        if tld_potencial in ["com", "net", "org", "gov", "edu", "mil", "arpa", "int"]:
            continue  # Si el segmento es un TLD válido, continuar buscando
        else:
            return tld_potencial  # Si no es un TLD válido, devolver este segmento como el TLD
    return dominio

In [10]:
# Aplicar la función obtener_tld a cada dominio único
tlds = [obtener_tld(dominio) for dominio in dominios_unicos]

# Crear un nuevo DataFrame solo con la columna domain_tld
df_tlds = pd.DataFrame({'domain_tld': tlds})

print(df_tlds)

           domain_tld
0    wunderground.com
1         dropbox.com
2           aoltw.net
3                home
4         mozilla.com
..                ...
172       localdomain
173     microsoft.com
174      verisign.com
175    stayonline.net
176          real.com

[177 rows x 1 columns]


### Utilizar el clasificador

In [11]:
df_resultado = clasificacion(df_tlds)

print(df_resultado)

           domain_tld  isDGA
0    wunderground.com      0
1         dropbox.com      1
2           aoltw.net      1
3                home      0
4         mozilla.com      0
..                ...    ...
172       localdomain      0
173     microsoft.com      1
174      verisign.com      0
175    stayonline.net      0
176          real.com      1

[177 rows x 2 columns]


c:\Users\diego\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [12]:

df_dga = df_resultado[df_resultado['isDGA'] == 1]

df_dga_unicos = df_dga.drop_duplicates()

print(len(df_dga_unicos))


32


### Filtrar con la lista

In [13]:
def verificar_tld(tld):
    """
    Función para verificar si un TLD se encuentra en una lista de TLDs proporcionada en un archivo CSV.

    Args:
    - tld (str): El TLD que se desea verificar.

    Returns:
    - int: 0 si el TLD está en la lista, 1 si no está.
    """
    # Ruta al archivo CSV que contiene la lista de TLDs
    archivo_tlds = 'top-1m.csv'

    # Intentamos abrir el archivo y verificar si el TLD está en la lista
    try:
        with open(archivo_tlds, 'r') as f:
            for linea in f:
                if linea.strip().endswith(tld.lower()):
                    return 0  # El TLD está en la lista
            return 1  # El TLD no está en la lista
    except FileNotFoundError:
        print("Error: No se pudo encontrar el archivo de lista de TLDs.")
        return -1 

In [14]:
# Crear una copia del DataFrame df_dga_unicos
df_dga_unicos_copia = df_dga_unicos.copy()

df_dga_unicos_copia['en_lista'] = df_dga_unicos_copia['domain_tld'].apply(verificar_tld)

df_en_lista = df_dga_unicos_copia[df_dga_unicos_copia['en_lista'] == 0]

print(len(df_en_lista))

df_en_lista_sin_duplicados = df_en_lista.drop_duplicates(subset='domain_tld')

print(df_en_lista_sin_duplicados)



22
         domain_tld  isDGA  en_lista
1       dropbox.com      1         0
5    metasploit.com      1         0
25      windows.com      1         0
48   phpmyadmin.net      1         0
51    microsoft.com      1         0
60   postgresql.org      1         0
62      freepbx.org      1         0
78       flickr.com      1         0
83     facebook.com      1         0
87       xmarks.com      1         0
92        adams.net      1         0
95     clarkson.edu      1         0
98         wisc.edu      1         0
100    easynews.com      1         0
101    bluehost.com      1         0
106      kernel.org      1         0
108   liquidweb.com      1         0
114  team-cymru.org      1         0
117    xmission.com      1         0
127       apple.com      1         0
144    msftncsi.com      1         0
176        real.com      1         0


In [15]:
import whois
from datetime import datetime

def obtener_fecha_creacion_tld(tld):

    try:
        # Realizar la consulta WHOIS para el TLD proporcionado
        w = whois.whois(tld)

#Obtener la fecha de creación desde el objeto WHOIS
        fecha_creacion = w.creation_date

#La fecha de creación puede ser una lista o un solo valor, dependiendo del TLD
        if isinstance(fecha_creacion, list):
            fecha_creacion = fecha_creacion[0]

#Formatear la fecha de creación como string en formato 'YYYY-MM-DD'
        if fecha_creacion:
            if isinstance(fecha_creacion, datetime):
                return fecha_creacion.strftime('%Y-%m-%d')
            else:
                return str(fecha_creacion)
        else:
            return "No se pudo encontrar la fecha de creación."
    except Exception as e:
        return "Error al buscar la fecha de creación: " + str(e)

In [17]:
df_en_lista_sin_duplicados['fecha_creacion'] = df_en_lista_sin_duplicados['domain_tld'].apply(obtener_fecha_creacion_tld)

# Mostrar el DataFrame con la fecha de creación
print(df_en_lista_sin_duplicados)

         domain_tld  isDGA  en_lista fecha_creacion
1       dropbox.com      1         0     1995-06-28
5    metasploit.com      1         0     2003-06-10
25      windows.com      1         0     1995-09-11
48   phpmyadmin.net      1         0     2002-04-02
51    microsoft.com      1         0     1991-05-02
60   postgresql.org      1         0     1996-10-22
62      freepbx.org      1         0     2005-10-11
78       flickr.com      1         0     2003-11-22
83     facebook.com      1         0     1997-03-29
87       xmarks.com      1         0     2003-12-29
92        adams.net      1         0     1994-12-23
95     clarkson.edu      1         0     1986-11-21
98         wisc.edu      1         0     1985-09-30
100    easynews.com      1         0     1996-10-31
101    bluehost.com      1         0     2002-11-15
106      kernel.org      1         0     1997-03-07
108   liquidweb.com      1         0     1997-08-05
114  team-cymru.org      1         0     2004-08-04
117    xmiss

### Dominios sospechosos

Se puede confirmar como dominio sospechoso:
- 11
- 22

### Prompt utilizados

aqui